In [17]:
import numpy as np
import string
import os
import math
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from natsort import natsorted
from nltk.corpus import stopwords 
import pandas as pd
# nltk.download('stopwords')
stop_words = stopwords.words('english')
folder_names = ["onFile"]

In [18]:
def read_file(filename):
    with open(filename, 'r', encoding ="ascii", errors ="surrogateescape") as f:
        stuff = f.read()
 
    f.close()
     
    return stuff

In [19]:
def tokenze_and_remove_stopwords(final_string):
    file_content = final_string
    tokens_words = nltk.word_tokenize(file_content)
    #print (tokens_words)
    
    #print words except "in" and "to" and "where"
    tokenize_words_without_stopwords = []
    for word in tokens_words:
        if word.lower() not in stop_words:
                tokenize_words_without_stopwords.append(word.lower())
        elif word.lower() == "in"  or word.lower() == "to" or word.lower() == "where" in stop_words:
                tokenize_words_without_stopwords.append(word.lower())
    return (tokenize_words_without_stopwords)

In [20]:
def positional_indexing():
 
    # Initialize the stemmer.
    #stemmer = PorterStemmer()

    # Initialize the file no.
    fileno = 1

    # Initialize the dictionary.
    pos_index = {}

    # Initialize the file mapping (fileno -> file name).
    file_map = {}

    for folder_name in folder_names:

        # Open files.
        file_names = natsorted(os.listdir("Files/" + folder_name))

        # For every file.
        for file_name in file_names:

            # Read file contents.
            stuff = read_file("Files/" + folder_name + "/" + file_name)

            # This is the list of words in order of the text.
            # We need to preserve the order because we require positions.
            # 'preprocessing' function does some basic punctuation removal,
            # stopword removal etc.
            final_token_list = tokenze_and_remove_stopwords(stuff)
            #print(final_token_list) 
            #بتطبع هنا واتاكد إن الفايلز شغالة تمام
            # For position and term in the tokens.
            for pos, term in enumerate(final_token_list):

                        # First stem the term.
                       # term = stemmer.stem(term)

                        # If term already exists in the positional index dictionary.
                        if term in pos_index:

                            # Increment total freq by 1.
                            pos_index[term][0] = pos_index[term][0] + 1

                            # Check if the term has existed in that DocID before.
                            if fileno in pos_index[term][1]:
                                pos_index[term][1][fileno].append(pos+1)

                            else:
                                pos_index[term][1][fileno] = [pos+1]

                        # If term does not exist in the positional index dictionary
                        # (first encounter).
                        else:

                            # Initialize the list.
                            pos_index[term] = []
                            # The total frequency is 1.
                            pos_index[term].append(1)
                            # The postings list is initially empty.
                            pos_index[term].append({})     
                            # Add doc ID to postings list.
                            pos_index[term][1][fileno] = [pos+1]

            # Map the file no. to the file name.
            file_map[fileno] = "Files/" + folder_name + "/" + file_name

            # Increment the file no. counter for document ID mapping             
            fileno += 1
        
        print("--------------------------------")
        print(">>>>>> Positional Index <<<<<< \n")
        print("Item: [ item frequancy, { files: [positions] } ] \n")
        for x, y in pos_index.items():
            print(x,y) #x y3ni term we y[0] y3ni el frequency 
        #return pos_index #lw 3awez arg3 el poitional index kolha mara w7da fe dict
        print("--------------------------------")
        
        # جزء الـ query اهووووووووووووووه
        #sample_pos_idx = pos_index['mercy']
       
        #print(sample_pos_idx)
        #file_list = sample_pos_idx[1]
        #print("Filename, [Positions]")
        #for fileno, positions in file_list.items():
        #    print(file_map[fileno], positions)
        
        query =input("Hello, Please insert your query:")
        
        final_list =[ [] for i in range(10) ]
        try:
            for word in query.lower().split():
                for key in pos_index[word][1].keys():
                    if final_list[key-1] !=[]:
                        if final_list[key-1][-1] == pos_index[word][1][key][0]-1:
                            final_list[key-1].append(pos_index[word][1][key][0])
                    else:
                        final_list[key-1].append(pos_index[word][1][key][0])
        except:
            print("Not Exist!!")
        #print(final_list)
        for position,inlist in enumerate(final_list, start=1):
        #    print(position,inlist)
            if len(inlist) == len(query.split()):
                print(position)
        return pos_index #lw 3awez arg3 el poitional index kolha mara w7da fe dict


In [29]:
sett={}
sett=set(sett)
Documents=[]
print("Words After Tokenization! \n")
for folder_name in folder_names:
 
    # Open files.
 
    file_names = natsorted(os.listdir("Files/" + folder_name))
 
    # For every file.
    for file_name in file_names:
 
        # Read file contents.
        stuff = read_file("Files/" + folder_name + "/" + file_name)
        
        # This is the list of words in order of the text.
        # We need to preserve the order because we require positions.
        # 'preprocessing' function does some basic punctuation removal,
        # stopword removal etc.
        final_token_list = tokenze_and_remove_stopwords(stuff) 
        Documents.append(final_token_list)# 
        for word in final_token_list:
            if word not in sett:
                sett.add(word) #dh elly feh el tokens
#print(Documents)     #--> list of documents that were toknized <-- dh elly feh tokens of files            
for word in sett:
    print(word+"\n")
# #pos_index = positional_indexing()
# #for x, y in pos_index.items():
# #        print(x, y)
PosIndexing = positional_indexing() #el positional index 
all_words=[]
for doc in Documents:
    for word in doc:
        all_words.append(word)

def get_term_freq(doc):
    words_found=dict.fromkeys(all_words,0)
    for word in doc:
        words_found[word] +=1
    return dict(sorted(words_found.items()))
term_freq=pd.DataFrame(get_term_freq(Documents[0]).values(),index=get_term_freq(Documents[0]).keys())
for i in range(1,len(Documents)):
    term_freq[i]=get_term_freq(Documents[i]).values()
term_freq.columns=['doc'+str(i) for i in range(1,11)]
print('='*108,' '*50,'\nThis is term frequency\n')
print(term_freq,"\n")
def get_weighted_term_freq(x):
    if x> 0:
        return int(math.log(x) +1)
    return 0

print('='*108,' '*50,'\nWeighted term frequency =log(tf)+1\n')
for i in range(1,len(Documents)+1):
    term_freq['doc'+str(i)]=term_freq['doc'+str(i)].apply(get_weighted_term_freq)
print(term_freq,"\n")
print('='*108,' '*50,'\ninverse document frequency\n')
tfd=pd.DataFrame(columns=('df','idf'))
for i in range(len(term_freq)):
    frequency=term_freq.iloc[i].values.sum() # (iloc) btgib el position bta3 elklma fe eldataframe
    tfd.loc[i,'df']=frequency
    tfd.loc[i,'idf']=math.log10(10/ float(frequency))

tfd.index=term_freq.index
print(tfd)

print('='*108,' '*50,'\ntf-idf\n')
term_freq_inve_doc_freq=term_freq.multiply(tfd['idf'],axis=0)
print(term_freq_inve_doc_freq)
print('='*108,' '*50,'\nDocument length\n')
document_length=pd.DataFrame()
def get_docs_length(col):
    return np.sqrt(term_freq_inve_doc_freq[col].apply(lambda x:x**2).sum())

for column in term_freq_inve_doc_freq.columns:
    document_length.loc['length',column+'_length']=get_docs_length(column)

print('\n',document_length)
print('='*108,' '*50,'\nNormalized tf-idf\n')
normalized_term_freq_idf=pd.DataFrame()
def get_normalized(col,x):
    try:
        return x / document_length[col+'_length'].values[0]
    except:
        return 0
for column in term_freq_inve_doc_freq.columns:   
    normalized_term_freq_idf[column] =term_freq_inve_doc_freq[column].apply(lambda x: get_normalized(column,x)) 

print(normalized_term_freq_idf)   


Words After Tokenization! 

worser

tread

caeser

where

calpurnia

to

cleopatra

fools

in

angels

rush

antony

mercy

brutus

fear

--------------------------------
>>>>>> Positional Index <<<<<< 

Item: [ item frequancy, { files: [positions] } ] 

antony [3, {1: [1], 2: [1], 6: [1]}]
brutus [3, {1: [2], 2: [2], 4: [1]}]
caeser [5, {1: [3], 2: [3], 4: [2], 5: [1], 6: [2]}]
cleopatra [1, {1: [4]}]
mercy [5, {1: [5], 3: [1], 4: [3], 5: [2], 6: [3]}]
worser [4, {1: [6], 3: [2], 4: [4], 5: [3]}]
calpurnia [1, {2: [4]}]
angels [3, {7: [1], 8: [1], 9: [1]}]
fools [4, {7: [2], 8: [2], 9: [2], 10: [1]}]
fear [3, {7: [3], 8: [3], 10: [2]}]
in [4, {7: [4], 8: [4], 9: [3], 10: [3]}]
rush [4, {7: [5], 8: [5], 9: [4], 10: [4]}]
to [4, {7: [6], 8: [6], 9: [5], 10: [5]}]
tread [4, {7: [7], 8: [7], 9: [6], 10: [6]}]
where [4, {7: [8], 8: [8], 9: [7], 10: [7]}]
--------------------------------


Hello, Please insert your query: in


7
8
9
10
This is term frequency

           doc1  doc2  doc3  doc4  doc5  doc6  doc7  doc8  doc9  doc10
angels        0     0     0     0     0     0     1     1     1      0
antony        1     1     0     0     0     1     0     0     0      0
brutus        1     1     0     1     0     0     0     0     0      0
caeser        1     1     0     1     1     1     0     0     0      0
calpurnia     0     1     0     0     0     0     0     0     0      0
cleopatra     1     0     0     0     0     0     0     0     0      0
fear          0     0     0     0     0     0     1     1     0      1
fools         0     0     0     0     0     0     1     1     1      1
in            0     0     0     0     0     0     1     1     1      1
mercy         1     0     1     1     1     1     0     0     0      0
rush          0     0     0     0     0     0     1     1     1      1
to            0     0     0     0     0     0     1     1     1      1
tread         0     0     0     0     0     

Enter query ,please:
 antony brutus



Query
         tf  w_tf       idf    tf_idf      norm
antony   1   1.0  0.522879  0.522879  0.707107
brutus   1   1.0  0.522879  0.522879  0.707107

query length :
 0.7394622130520805

product (query*match doc)

             doc1      doc2
antony  0.269196  0.288939
brutus  0.269196  0.288939

product sum
doc1    0.538393
doc2    0.577877
dtype: float64

cosine similarity:
doc1    0.538393
doc2    0.577877
dtype: float64

returned docs
doc2 doc1 

C:\Users\USER\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
